In [2]:
from IPython.display import HTML

### Q1. 

What is happening here? Hazard a guess as to why these id's are the same.

In [3]:
e=[1,2,3,4]
d=1
id(e[0]), id(d)

(4297514912, 4297514912)

Python uses name binding and caches furthermore integers up to a certain number as objects. Thus, e[0] and d have the same id (i.e. like a C pointer). Consider in comparison this example, where the result is different:

In [4]:
a = [512, 257, 259]
b = 512

id(a[0]), id(b)


(4362073104, 4362074416)

In [5]:
HTML('<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=e%3D%5B1,2,3,4%5D%0Aprint(%22e%22,e%29&origin=opt-frontend.js&cumulative=false&heapPrimitives=true&textReferences=false&py=3&rawInputLstJSON=%5B%5D&curInstr=0&codeDivWidth=350&codeDivHeight=400"> </iframe>')

### Q2.

Here we work through an example taken from SICP, in which we try and write a bank account withdrawal system

In [6]:
def make_withdraw1(balance):
    """Return a withdraw function that draws down balance with each call."""
    def withdraw(amount):#you are ok if you use balance as a read-only property
        if amount > balance:
            return 'Insufficient funds'
        return balance - amount
    return withdraw

Lets make a 5 buck withdrawal

In [7]:
wdfrom20=make_withdraw1(20)
wdfrom20(5)

15

Lets do it again

In [8]:
wdfrom20(5)

15

This is not quite what we want. We'd want it to go down to 10 on a repeated withdrawal. Remember, writing it like this, we have no access to `balance` directly. In other words, we want `balance` that was captured from the enclosing frame to be *changeable*, not unchangeable. 

Lets try something simple. Even though `balance` was captured from the enclosing frame, lets try reassigning it in `withdraw`. So, write a new `make_withdraw2` in which `withdraw` has this re-assignment. What happens when you do this?

In [9]:
def make_withdraw2(balance):
    def withdraw(amount):#you are ok if you use balance as a read-only property
        if amount > balance:
            return 'Insufficient funds'
        balance -= amount 
        return balance
    return withdraw


In [10]:
wdfrom20_2=make_withdraw2(20)
wdfrom20_2(5)

UnboundLocalError: local variable 'balance' referenced before assignment

If you did it right, you should have got an error. As the Python Execution Model says:

> If a name binding operation occurs anywhere within a code block, all uses of the name within the block are treated as references to the current block. This can lead to errors when a name is used within a block before it is bound. This rule is subtle. Python lacks declarations and allows name binding operations to occur anywhere within a code block. The local variables of a code block can be determined by scanning the entire text of the block for name binding operations.

What has happened is that `balance` is now being treated as a local variable in `withdraw`, since the assignment has created a "name" for it. And then it does not have a value.

### Q3.

We need to somehow tell Python that `balance` is NOT a local variable. The way to do this is the `nonlocal` keyword. Write a function `make_withdraw` using  

`nonlocal balance` 

as the first line of the inner `withdraw` function. 

In [11]:
def make_withdraw(balance):
    def withdraw(amount):#you are ok if you use balance as a read-only property
        nonlocal balance
        if amount > balance:
            return 'Insufficient funds'
        balance -= amount 
        return balance
    return withdraw


In [12]:
from20_correct = make_withdraw(20)
from20_correct(5)

15

In [13]:
from20_correct(5)

10

### Q4.

Go to http://pythontutor.com/index.html, choose Python 3.3 as the language, and visualize your code. Embed the code from above in there. Pay attention to the variable `balance` in `f1: make_withdraw`.

In [38]:
# Somehow this doesn't work
HTML('<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=def+make_withdraw(balance%29%3A%0A++++def+withdraw(amount%29%3A%23you+are+ok+if+you+use+balance+as+a+read-only+property%0A++++++++nonlocal+balance%0A++++++++if+amount+%3E+balance%3A%0A++++++++++++return+'Insufficient+funds'%0A++++++++balance+-%3D+amount+%0A++++++++return+balance%0A++++return+withdraw%0A++++%0Afrom20_correct+%3D+make_withdraw(20%29%0Afrom20_correct(5%29%0Afrom20_correct(5%29&origin=opt-frontend.js&cumulative=false&heapPrimitives=false&textReferences=false&py=3&rawInputLstJSON=%5B%5D&curInstr=0&codeDivWidth=350&codeDivHeight=400"> </iframe>')

SyntaxError: invalid syntax (<ipython-input-38-1eca108cd682>, line 2)

see under http://pythontutor.com/visualize.html#code=def+make_withdraw(balance%29%3A%0A++++def+withdraw(amount%29%3A%23you+are+ok+if+you+use+balance+as+a+read-only+property%0A++++++++nonlocal+balance%0A++++++++if+amount+%3E+balance%3A%0A++++++++++++return+'Insufficient+funds'%0A++++++++balance+-%3D+amount+%0A++++++++return+balance%0A++++return+withdraw%0A++++%0Afrom20_correct+%3D+make_withdraw(20%29%0Afrom20_correct(5%29%0Afrom20_correct(5%29&mode=display&origin=opt-frontend.js&cumulative=false&heapPrimitives=false&textReferences=false&py=3&rawInputLstJSON=%5B%5D&curInstr=0

Notice that using the "constructor" `make_withdraw` twice creates two different "objects".

In [39]:
wd=make_withdraw(20)
print(wd(5))
wd2=make_withdraw(100)
print(wd2(5))

15
95


In [40]:
id(wd),id(wd2)

(4365300056, 4365091024)

Futhermore, notice that even though we cant access `balance` directly, we can change its value using the "created". So we now have a way to create state, as well as a way to change it. This is a very simple object system.

### Q5.

Use the decorator `timer` which we developed in the lecture to time how long it takes a function to run. Use this timer to plot a graph of the amount of time it takes to fill multiple arrays of size 100, 1000, 10000, 100000, 1000000, 10000000 with integers upto and not including that number.

In [41]:
#the decorator, slightly modified
import time
def time_returner(f):
    def inner(*args):
        t0 = time.time()
        f(*args)
        elapsed = time.time() - t0
        return elapsed
    return inner

In [42]:
#your code here
@time_returner
def fill_array(N):
    arr = []
    for i in range(N):
        arr.append(i)
    return arr

In [43]:
#your code here
for i in range(2, 8):
    N = 10**i
    print('filled array size={} in {} sec:'.format(N, fill_array(N)))

filled array size=100 in 2.193450927734375e-05 sec:
filled array size=1000 in 0.00017189979553222656 sec:
filled array size=10000 in 0.0012209415435791016 sec:
filled array size=100000 in 0.019067049026489258 sec:
filled array size=1000000 in 0.16365408897399902 sec:
filled array size=10000000 in 1.2914009094238281 sec:


### Q6.

A decorator can have arguments.

If you want to create a decorator with arguments, such as for example, a state that is global such as a counter, what you need to do is to capture the state variable as a closure inside the decorating function.

Here is the skeleton for that:

```python
def call_to_create_a_decorator(state_arguments):
    def this_is_the_decorator_function(user_function):
        def inner_function(*args):
            #wrapping stuff here
            output = user_function(*args)
            #pehaps more wrapping here
            return output # or something else
        return inner_function
    return this_is_the_decorator_function
```

In [44]:
def call_counter(count_dictionary):
    def call_counter_decorator(func):
        def inner(*args, **kwargs):
            n = func.__name__
            #your code here
            
            # check if already key for func exists and inc call counter
            if n not in count_dictionary.keys():
                count_dictionary[n] = 0
            count_dictionary[n] += 1
            
            output = func(*args)
            # call func
            return output
        return inner
    return call_counter_decorator


In [45]:
storage={}

In [46]:
@call_counter(storage)
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-2) + fib_recursive(n-1)

In [47]:
@call_counter(storage)
def fact(n):
    if n<=1:
        return 1
    else:
        return n*fact(n-1)

Run the following code:

```python
for i in [7, 13, 29]:
    fib_recursive(i)
    fact(i)
    print(i, storage['fib_recursive'], storage['fact'])
```

In [48]:
for i in [7, 13, 29]:
    fib_recursive(i)
    fact(i)
    print(i, storage['fib_recursive'], storage['fact'])

7 25 7
13 490 20
29 1028947 49


Enter the last two numbers (as below) in the google form http://goo.gl/forms/Fyv2PLiJdw.

In [49]:
print(storage['fib_recursive'], storage['fact'])

1028947 49
